# **Cold Spot SCANNER**

![](https://github.com/sagagugit/Cold-Spot-Scanner/blob/main/cold_spot_scanner.png?raw=true)

**About cold-spots in protein-protein interactions**

Cold spots are positions in proteins occupied by non-optimal amino acids. At cold-spot positions, several possible mutations lead to affinity enhancement [1]. We identified three possible scenarios how cold-spots could occur:


 (I) Cold-spots occur at positions where a WT amino acid does not interact with the binding partner, creating a **cavity**.


(II) Cold-spots occur at positions where a charged amino acid is buried in hydrophobic environment, exhibiting **unfavourable Charge-Hydrophobic (CH) interactions**. Eliminating the charged amino acid through a mutation enhances affinity.


(III) Cold-spots occur at positions where two amino acids of the same charge come close together, exibiting **unfavorable Same-Charge (SC) interactions**. Eliminating one of these charges through a mutation enhances affinity.


Previously, we searched for cold spots in the whole PDB and proved that cold spots are a general feature in evolution of protein-protein iteractions[2].


**About this Colab notebook**

This colab notebook allows you to identify cold spots due to the above three scenarios in your favorite protein-protein complex. Once the structure is uploaded, the notebook computes the cold spots, visualizes the cold spots in the structure of the complex, and creates a summary file for download.

**Instructions for using this Colab notebook**

Two options are possible for uploading the protein complex structure.

1)	**The complex structure is downloaded directly from the PDB**. Please input the "PDB ID" of the Protein complex and chain IDs to define the interface of the protein complex under the sub category "PDB ID and chains". If you mention chains that are not interacting, the program will generate an error message upon execution.  


2)	**The complex structure is uploaded from the user’s computer**. To enable users to upload their own complex, kindly remove the comment symbols (#) from all lines in the section labeled "Uploading the complex instead of PDB ID". Once uncommented, the user can upload their desired complex upon execution. **Before execution of the program**, specify its corresponding four-letter name and chains under the "PDB ID and chains" subcategory. Please ensure that the name assigned to the uploaded complex is limited to a four-letter format, similar to that of a PDB ID.



**Citing this work**


Please cite the articles

1)J. Shirian, O. Sharabi, and J. M. Shifman. 2016. “Cold-spots in Protein Binding.” Trends Biochem Sci, 41(9), Pp. 739-45.


2) Sagara N. S. Gurusinghe, Ben Oppenheiner, and Julia M. Shifman. 9/21/2022."Cold Spots are Universal in protein- protein interactions",  Protein Science, 31, 10, Pp. e4435.

   
Contact:
If you encounter any problems of errors with running this notebook please contact Sagara N.S. Gurushighe (sagara.gurusinghe@mail.huji.ac.il) or Julia M. Shifman (jshifman@mail.huji.ac.il).


 We recommend using either **Google Chrome** or **Mozilla Firefox** to effectively utilize and navigate through this Colab page.  

# Download the Cold Spot SCANNER

In [1]:
%%bash

cd /content/

if [ ! -f Cold-Spot-Scanner ]; then


    # delete the Cold-scanner/ directory if it already exists
    if [ -d "Cold-Spot-Scanner/" ]; then
        rm -rf Cold-Spot-Scanner/
    fi

    # download model
    git clone https://github.com/sagagugit/Cold-Spot-Scanner --quiet
    touch Cold-Spot-Scanner
fi

# Prepare input for the scan

In [2]:
#@title Install necessary packages
#Install Biopython
%%capture
!pip install Bio
!pip install py3Dmol
!pip install fileupload

In [3]:
#@title Selecting the path
%%capture
cd Cold-Spot-Scanner

Uploading the complex instead of PDB ID

In [4]:
# try:
#   from google.colab import drive
#   drive.mount('/content/drive')
#   from google.colab import files
#   uploaded = files.upload()
# except FileNotFoundError:
#   print("ERROR: \n Uploading was not successful. Please restart and try to upload the complex again")

In [5]:
#@title PDB ID and chains
#Please provide the 4 letter PDB ID of the protein protein complex
# Example shown below '1PXV', (Protein complex of cysteine protease with cysteine protease Inhibitor)
PDB = '8DAF'#@param {type:"string"}
#PDB =  'PDB'#@param {type:"string"}
#Please proivde the two chains below to define the interface of the protein complex
# Example shown below the two chains 'A:C' of the complex cysteine protease with cysteine protease Inhibitor
#chains = 'A:B'
chain_A =  'A'#@param {type:"string"}
chain_B =  'C'#@param {type:"string"}



# Run Cold Spot SCANNER and visualise the cold spots

In [6]:
#@title Running the cold spot scan
%%capture
input = PDB + '_' + chain_A + ':' + chain_B
f = open("PDB_and_chain", "w")
f.write(input)
f.close()
!python init_database.py x
!python setupPpiDb.py PDB_and_chain x
!cp /content/Cold-Spot-Scanner/results/cold_spots/Cold_spots_residues_due_to_CH_interactions.csv /content/Cold-Spot-Scanner
!cp /content/Cold-Spot-Scanner/results/cold_spots/Cold_spots_due_to_SC_interactions.csv /content/Cold-Spot-Scanner

In [7]:
try:
  #@title Visualise cold spots
  import pandas as pd
  df = pd.read_csv("Cold_spots_residues_due_to_CH_interactions.csv",skiprows=1)
  df1 = pd.read_csv("Cold_spots_due_to_SC_interactions.csv",skiprows=1)
  df = df.dropna()
  df1 = df1.dropna()
  df['Chain'] = df['Chain'].str.lstrip()
  df1['Chain'] = df1['Chain'].str.lstrip()
  import py3Dmol
  try:
    with open(PDB+"Sepereated_clusters.pdb") as ifile:
        system = "".join([x for x in ifile])
  except:
    pass
  with open(PDB+"_C.pdb") as ifilex:
      systemx = "".join([x for x in ifilex])

  view = py3Dmol.view(width=800, height=600)
  # view.addModelsAsFrames(system)
  # view.setStyle({'model': -1}, {"sphere": {'color': 'spectrum'}})
  view.addModelsAsFrames(systemx)
  view.setStyle({'model': -1}, {'cartoon': {'colorscheme': 'chain'}})
  # view.addStyle({'chain':df['x'],'resi':y},
  #               {'stick':{'colorscheme':'blueCarbon'}})
  for idx, row in df1.iterrows():
      chain = row['Chain']
      resi = row['Residue_number']
      view.setStyle({'chain': chain, 'resi': str(resi)}, {'stick': {'color': 'blue'}})
  for idx, row in df.iterrows():
      chain = row['Chain']
      resi = row['Residue_number']
      view.setStyle({'chain': chain, 'resi': str(resi)}, {'stick': {'color': 'yellow'}})
  view.zoomTo()
  view.show()
except FileNotFoundError:
  print("ERROR: \n Please give a valid PDB ID and Chain identifiers for that specific PDB ID. \n  If you upload a complex Kindly ensure that the name assigned to the uploaded complex is limited to a four-letter format, similar to that of a PDB ID")

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Download Results

In [8]:
cd /content/Cold-Spot-Scanner/results/

/content/Cold-Spot-Scanner/results


In [9]:
!rm /content/Cold-Spot-Scanner/results/cold_spots/Cold_spots_residues_due_to_CH_interactions.csv
!rm /content/Cold-Spot-Scanner/results/cold_spots/Cold_spots_due_to_SC_interactions.csv

In [10]:
from google.colab import files
import shutil
shutil.make_archive('cold_spots', 'zip', 'cold_spots')
files.download(f'cold_spots.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>